# Calculate current Wallet Value

In [8]:
from Feeds.infura_multichain import Web3Infura, get_checksum_address
from Feeds.etherscan import EtherscanClient
from Feeds.utility import *

from Feeds.coingecko import *
from Feeds.dexscreener import *

from datetime import datetime
import warnings
import urllib3

warnings.filterwarnings("ignore", category=urllib3.exceptions.InsecureRequestWarning)

In [9]:
ETHERSCAN_API_KEY = 'KR52G3163AU9696NCAZ74CGFKJZV7GIIG6'


addresses = {
    "sample_random_0": "0x0f000bFDD83597758011189898073653d97bf557",
    "sample_random_1": "0x5664773F216D8006D5cfEc4c9De324395d05b422",
    "sample_random_2": "0x1929d4d2b476f38ad06688c4b11fd01aacf4c9f5",
    "NeuralAI": "0x32B053F2CBA79F80ada5078cb6b305da92BDe6e1",
    "SharpAI": "0xaddb6dc7e2f7caea67621dd3ca2e8321ade33286",
    "INFRA": "0xe9EccDE9d26FCBB5e93F536CFC4510A7f46274f8",
    "KNDX": "0x7CA5af5bA3472AF6049F63c1AbC324475D44EFC1",
    "BUILD": "0x73454ACfdDb7a36A3cd8Eb171fBEa86c6a55E550",
    "TAOBOT": "0x49fb8ad7578148E17c3eF0C344CE23A66ed372C4",
    "Vmint": "0xD7B2C1a7F3c67fB0EA57a7ef29bC1F18D7bE3195",
    "CHAT": "0xBb3D7F42C58Abd83616Ad7C8C72473Ee46df2678",
}

**Get wallet current portfolio**

In [10]:
wallet_address = addresses["sample_random_0"]

In [14]:
def get_tokens(wallet_address):
    EtherscanClinet = EtherscanClient(api_key=ETHERSCAN_API_KEY)

    # transakcje inicjowane przez użytkownika, tutaj mamy wszystkie wysłane, otrzymane tokeny, uniswapy itp
    # tutaj mamy wszystkie transakcje ale bez symbolu tokena i contract address, które są potrzebne do odczytania dodatkowych informacji co to za token
    df_normal_transactions = EtherscanClinet.get_normal_transactions_by_address(address=wallet_address)

    # transfery tokenów z walleta na wallet (W2W), wpłaty przy użyciu kontraktów np zakup na Uniswap, w tym randomowe airdropy shitcoinów, scamowe przelewy itp
        # tutaj mamy te dodatkowe informacje jak symbol tokena lub contract address
    df_transfers_address = EtherscanClinet.get_token_transfers_by_address(address=wallet_address)

    # aby wyfiltrować shitcoiny bierzemy 'get_token_transfers_by_address' i filtrujemy tylko po hash z 'get_normal_transactions_by_address'
    # wtedy otrzymujemy transakcje tokenów zainicjowane przez użytkownika, więc nie są to jakieś scamy, aidropy itp tylko faktycznie dokonane inwestycjie (lub przelane W2W)
    valid_tokens = df_transfers_address[df_transfers_address['hash'].isin(df_normal_transactions['hash'])]['tokenSymbol'].unique().tolist()
    
    # pozostawiamy tylko transakcje na tych tokenach które były prawdopodbnie zainicjowane przez użytkownika
    data = df_transfers_address[df_transfers_address['tokenSymbol'].isin(valid_tokens)].copy()
    data[["tokenDecimal", "value"]] = data[["tokenDecimal", "value"]].astype(float)
    return data

df_tokens = get_tokens(wallet_address)
df_tokens.tail(4)

Fetching blockchain data from block 1...
last_block: 21020940, Num trans: 89
Fetching blockchain data from block 1...
last_block: 21020940, Num trans: 106


,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,tokenSymbol,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations
95,20977065,1729068923,0x079022a44b658b15eac3412a98558870bbfa2518a7d6...,68,0x05d5554470296e7fb2c892734d89edba947dec42e50e...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0x59a73bef0f729761bc3b9aa7f41872077e4bf300,0x0f000bfdd83597758011189898073653d97bf557,1.346669e+22,SECT BOT,SECT,18.0,38,241013,16457017411,165516,5966464,deprecated,66610
101,21006168,1729419803,0x560fd1f4a9a5fa9325777d630a470b2fde1ab4a9960d...,69,0x08e291354e4d8fdaf39a52f5149a4fcbe1b11bb24007...,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,0xf445adc37b8185eeee6cb8ac1628e59bda4b072b,0x0f000bfdd83597758011189898073653d97bf557,1.593048e+22,AR:AI,ARAI,18.0,3,291337,9152708346,204861,2312899,deprecated,37507
104,21020940,1729597955,0x17c320da370588643586b2b0bea3973fc1c9870df37d...,71,0x7d56fc9888ac89a7ab2c04fcc0215361d78949577147...,0x0f000bfdd83597758011189898073653d97bf557,0xf445adc37b8185eeee6cb8ac1628e59bda4b072b,0xf445adc37b8185eeee6cb8ac1628e59bda4b072b,6.372191e+20,AR:AI,ARAI,18.0,29,274999,7999941534,197163,2464780,deprecated,22735
105,21020940,1729597955,0x17c320da370588643586b2b0bea3973fc1c9870df37d...,71,0x7d56fc9888ac89a7ab2c04fcc0215361d78949577147...,0x0f000bfdd83597758011189898073653d97bf557,0xf445adc37b8185eeee6cb8ac1628e59bda4b072b,0x854e5cb095a654583677e87ae7c3c83e04e398a6,1.529326e+22,AR:AI,ARAI,18.0,29,274999,7999941534,197163,2464780,deprecated,22735


In [16]:
balances = {}
contract_address = {}

def update_balance(row):
    token = row['tokenSymbol']
    decimals = row['tokenDecimal']
    value = row['value'] / (10 ** decimals)  # Adjust for decimals
    
    # Initialize balance for the token if it doesn't exist
    if token not in balances:
        balances[token] = 0
        contract_address[token] = row['contractAddress']

    # Check if it's an inflow or outflow
    if row['to'].lower() == wallet_address.lower():
        balances[token] += value  # Inflow
    elif row['from'].lower() == wallet_address.lower():
        balances[token] -= value  # Outflow


df_tokens.apply(update_balance, axis=1)

for token, balance in balances.items():
     if balance != 0 and abs(balance) >= 0.0001:
        print(f"Token: {token}, Balance: {balance}, Contract Address: {contract_address[token]}")

Token: BDID, Balance: 30875.577890828477, Contract Address: 0x5016878159e84dadb05bb04135f3eac339ae201f
Token: TAOBOT, Balance: 464930.3688443876, Contract Address: 0x49fb8ad7578148e17c3ef0c344ce23a66ed372c4
Token: vMINT, Balance: 939840.9819134906, Contract Address: 0xd7b2c1a7f3c67fb0ea57a7ef29bc1f18d7be3195
Token: NEURAL, Balance: 4530.369081439087, Contract Address: 0x32b053f2cba79f80ada5078cb6b305da92bde6e1
Token: INFRA, Balance: 615.243215387, Contract Address: 0xe9eccde9d26fcbb5e93f536cfc4510a7f46274f8
Token: CHAT, Balance: 56006.13578435295, Contract Address: 0xbb3d7f42c58abd83616ad7c8c72473ee46df2678
Token: SECT, Balance: 13466.68802587581, Contract Address: 0x59a73bef0f729761bc3b9aa7f41872077e4bf300


**Get token balance using INFURA**

I do not know excatly why but ehterscan API returns sometimes less precise data, and Infura is more reliable here regarding current token balance (probably due to directio nodeconnection)

In [19]:
selected_chain = 'ethereum'  # Example for Polygon chain


infura_client = Web3Infura(chain=selected_chain)

token_balance = infura_client.get_token_balance(
    get_checksum_address(0xe9eccde9d26fcbb5e93f536cfc4510a7f46274f8),
    get_checksum_address(wallet_address),
    )
token_balance

Decimal('577.743215387')

**Get current coingecko price**

In [17]:
price = get_token_price_coingecko(
    token_address="0x49fb8ad7578148e17c3ef0c344ce23a66ed372c4"
    )

price

0.062292